<a href="https://colab.research.google.com/github/guymatz/csci-49376/blob/develop/csci_49376_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
!pip install pyspark

In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import functions
# Create a Spark Session
ss = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
edges = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/edges.tsv", header=True, sep="\t")
nodes = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/nodes.tsv", header=True, sep="\t")

In [ ]:
print(f"Edges: {edges.count()}")
print(f"Nodes: {nodes.count()}")

Edges: 1292203
Nodes: 23036


In [ ]:
# For each drug, compute the number of genes
# and the number of diseases associated with the
# drug. Output results with top 5 number of genes in a
# descending order

In [ ]:
compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Gene') 
        |
        edges.target.startswith('Disease')
    )
)

In [40]:
compounds.sample(.01).show()

+-----------------+--------+-------------------+
|           source|metaedge|             target|
+-----------------+--------+-------------------+
|Compound::DB00398|     CtD| Disease::DOID:1781|
|Compound::DB00331|     CtD|Disease::DOID:14221|
|Compound::DB01024|     CtD|  Disease::DOID:418|
|Compound::DB01006|     CtD| Disease::DOID:1612|
|Compound::DB01042|     CtD| Disease::DOID:1612|
|Compound::DB00915|     CbG|         Gene::6582|
|Compound::DB00307|     CbG|         Gene::6258|
|Compound::DB00281|     CbG|         Gene::1544|
|Compound::DB00590|     CbG|          Gene::146|
|Compound::DB01221|     CbG|         Gene::6869|
|Compound::DB01239|     CbG|         Gene::1813|
|Compound::DB00731|     CbG|         Gene::6833|
|Compound::DB00818|     CbG|        Gene::54576|
|Compound::DB01156|     CbG|         Gene::1548|
|Compound::DB00511|     CbG|          Gene::477|
|Compound::DB00174|     CbG|        Gene::80150|
|Compound::DB01029|     CbG|         Gene::5742|
|Compound::DB01587| 

In [38]:
c3 = c2.drop('metaedge', 'target')
c3.show()

+-----------------+-----------+
|           source|target_type|
+-----------------+-----------+
|Compound::DB00997|    Disease|
|Compound::DB00206|    Disease|
|Compound::DB00960|    Disease|
|Compound::DB00665|    Disease|
|Compound::DB00290|    Disease|
|Compound::DB01232|    Disease|
|Compound::DB00555|    Disease|
|Compound::DB00444|    Disease|
|Compound::DB00860|    Disease|
|Compound::DB00635|    Disease|
|Compound::DB00985|    Disease|
|Compound::DB00808|    Disease|
|Compound::DB01244|    Disease|
|Compound::DB00888|    Disease|
|Compound::DB00591|    Disease|
|Compound::DB00530|    Disease|
|Compound::DB00191|    Disease|
|Compound::DB00620|    Disease|
|Compound::DB00853|    Disease|
|Compound::DB00654|    Disease|
+-----------------+-----------+
only showing top 20 rows



In [60]:
filtered_compounds = compounds.rdd.map(lambda x: ((x.source,"G"), 1) if x.target.startswith("Gene") else ((x.source, "D"), 1))
filtered_compounds.take(10)

[(('Compound::DB00997', 'D'), 1),
 (('Compound::DB00206', 'D'), 1),
 (('Compound::DB00960', 'D'), 1),
 (('Compound::DB00665', 'D'), 1),
 (('Compound::DB00290', 'D'), 1),
 (('Compound::DB01232', 'D'), 1),
 (('Compound::DB00555', 'D'), 1),
 (('Compound::DB00444', 'D'), 1),
 (('Compound::DB00860', 'D'), 1),
 (('Compound::DB00635', 'D'), 1)]

In [61]:
filtered_compound_counts = filtered_compounds.reduceByKey(lambda a,b:a + b)
#print(type(filtered_compound_counts))
filtered_compound_counts.take(5)


[(('Compound::DB00960', 'D'), 1),
 (('Compound::DB00665', 'D'), 1),
 (('Compound::DB01232', 'D'), 1),
 (('Compound::DB00555', 'D'), 3),
 (('Compound::DB00635', 'D'), 17)]

In [65]:
sorted_count = filtered_compound_counts.sortBy(lambda x: x[1], ascending=False)
sorted_count.take(5)

[(('Compound::DB08865', 'G'), 585),
 (('Compound::DB01254', 'G'), 564),
 (('Compound::DB00997', 'G'), 532),
 (('Compound::DB00570', 'G'), 523),
 (('Compound::DB00390', 'G'), 522)]

In [ ]:
# Q2: Compute the number of diseases associated
# with 1, 2, 3, …, n drugs. Output results with the top
# 5 number of diseases in a descending order.
disease_compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Disease')
    )

)

In [ ]:
filtered_disease_compounds = disease_compounds.rdd.map(lambda x: (x.target,1))
filtered_disease_compounds.take(5)
filtered_disease_compound_counts = filtered_disease_compounds.reduceByKey(lambda a,b:a + b)
sorted_disease_count = filtered_disease_compound_counts.sortBy(lambda x: x[1], ascending=False)
sorted_disease_count.take(5)

[('Disease::DOID:10763', 70),
 ('Disease::DOID:2531', 53),
 ('Disease::DOID:1612', 43),
 ('Disease::DOID:2841', 40),
 ('Disease::DOID:3393', 38)]

In [ ]:
# Q3: Get the name of drugs that have the top 5 number of genes. Output the results.